In [659]:
import numpy as np
import pandas as pd

In [660]:
movies=pd.read_csv('tmdb_5000_movies.csv')

In [661]:
from sklearn.preprocessing import LabelEncoder
movies.iloc[:,1]=LabelEncoder().fit_transform(movies.iloc[:,1])#converting to nubmer for genric type

In [662]:
new_movie=movies['genres'].value_counts()
new_movie=new_movie[new_movie>2]

In [663]:
new_rate=movies['vote_average'].value_counts()
new_rate=new_rate[new_rate>20]
new_rate.head()

6.5    216
6.0    216
6.7    213
6.3    207
6.1    201
Name: vote_average, dtype: int64

In [664]:
movies=movies[movies['genres'].isin(new_movie.index)][['genres','title','vote_average','vote_count']]
movies.head()
#final_movies=movies.merge(new_movie,on='genres')[['genres','title','vote_average','vote_count','title count']]

,genres,title,vote_average,vote_count
0,608,Avatar,7.2,11800
1,143,Pirates of the Caribbean: At World's End,6.9,4500
2,674,Spectre,6.3,4466
3,791,The Dark Knight Rises,7.6,9106
4,677,John Carter,6.1,2124


In [665]:
movies=movies[movies['vote_average'].isin(new_rate.index)]
movies.head()

,genres,title,vote_average,vote_count
0,608,Avatar,7.2,11800
1,143,Pirates of the Caribbean: At World's End,6.9,4500
2,674,Spectre,6.3,4466
3,791,The Dark Knight Rises,7.6,9106
4,677,John Carter,6.1,2124


In [666]:
new_movies=movies.groupby('genres').agg('count')['vote_count'].reset_index().rename({'vote_count':'count'},axis=1)
movies=movies.merge(new_movies,on='genres')
movies.head()

,genres,title,vote_average,vote_count,count
0,608,Avatar,7.2,11800,6
1,608,Man of Steel,6.5,6359,6
2,608,X-Men: Days of Future Past,7.5,6032,6
3,608,Superman,6.9,1022,6
4,608,Superman II,6.5,629,6


In [667]:
movies=movies[movies['vote_average']>5.5]
movies=movies[movies['vote_count']>100]
movies.head()

,genres,title,vote_average,vote_count,count
0,608,Avatar,7.2,11800,6
1,608,Man of Steel,6.5,6359,6
2,608,X-Men: Days of Future Past,7.5,6032,6
3,608,Superman,6.9,1022,6
4,608,Superman II,6.5,629,6


In [668]:
final_pivot=movies.pivot_table(values='vote_average',index='title',columns='genres').fillna(0)
final_pivot.head()

genres,4,8,15,16,28,34,43,47,57,70,...,1122,1134,1137,1154,1156,1160,1162,1165,1166,1174
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Rounds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Years a Slave,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [669]:
from scipy.sparse import csr_matrix
movies_sparse=csr_matrix(final_pivot)

In [670]:
from sklearn.neighbors import NearestNeighbors
nn=NearestNeighbors(metric='cosine',algorithm='brute')
nn.fit(movies_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [671]:
np.where(final_pivot.index=='X-Men')[0][0]

1962

In [672]:
distance,suggestion=nn.kneighbors(final_pivot.iloc[,:].values.reshape(1,-1),n_neighbors=6)

In [673]:
suggestion=final_pivot.index[suggestion].ravel()

In [674]:
for i in range(len(suggestion)):
    print([suggestion[i]])

['The Lost World: Jurassic Park']
['Journey 2: The Mysterious Island']
['The Hunger Games: Catching Fire']
['Divergent']
['X-Men']
['Captain America: Civil War']
